# T09 : Évaluation Comparative Robuste (ML vs Règles vs RL)

## Objectif
Ce notebook finale vise à comparer les performances de trois approches de trading sur le jeu de données de test (2024), qui n'a jamais été vu par les modèles :

1.  **Approche Règles (Rule-Based)** : Stratégie classique EMA + RSI.
2.  **Approche Machine Learning (ML)** : Random Forest (Classification de la direction).
3.  **Approche Reinforcement Learning (RL)** : Simulation de l'environnement (Agent PPO basique/aléatoire en l'absence de librairie RL avancée).

## Méthodologie
- **Données** : GBP/USD M15 (2022-2024).
- **Train** : 2022 | **Val** : 2023 | **Test** : 2024 (Strictement séparés).
- **Coûts** : Spread + Commission simulés (0.0002 ou 2 pips).
- **Métriques** : Sharpe Ratio, Max Drawdown, Profit Factor, Win Rate.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import os

# Configuration visuelle
plt.style.use('seaborn-v0_8-pastel')
sns.set_theme(style="whitegrid")
COLORS = {'Train': '#EAE7DC', 'Val': '#D8C3A5', 'Test': '#8E8D8A', 'Profit': '#4E8D7C', 'Loss': '#E85A4F'}

# Modification du path pour charger les fichiers séparés
DATA_DIR = '../data/features/'

## 1. Chargement et Préparation des Données

In [ ]:
def load_data(data_dir):
    files = ['GBPUSD_M15_2022_features.csv', 'GBPUSD_M15_2023_features.csv', 'GBPUSD_M15_2024_features.csv']
    dfs = []
    for f in files:
        path = os.path.join(data_dir, f)
        if os.path.exists(path):
            df_year = pd.read_csv(path, parse_dates=['timestamp'], index_col='timestamp')
            dfs.append(df_year)
        else:
            print(f"Warning: Fichier manquant {path}")
            
    if not dfs:
        raise FileNotFoundError("Aucun fichier de données trouvé")
        
    df = pd.concat(dfs)
    df.sort_index(inplace=True)
    
    # Création de la target (futur immédiat) pour le ML
    # Shift(-1) : le return de la prochaine bougie
    df['target_return'] = df['close_15m'].shift(-1) - df['close_15m']
    df['target'] = (df['target_return'] > 0).astype(int)
    
    # Nettoyage NaN
    df.dropna(inplace=True)
    
    return df

df_full = load_data(DATA_DIR)
print(f"Données chargées (Total): {df_full.shape}")

# Split par année (automatique grâce à l'index datetime)
train_data = df_full.loc['2022']
val_data = df_full.loc['2023']
test_data = df_full.loc['2024']

print(f"Train (2022): {train_data.shape}")
print(f"Val (2023)  : {val_data.shape}")
print(f"Test (2024) : {test_data.shape}")

## 2. Moteur de Backtest Vectorisé
Ce moteur simule l'exécution des trades avec coûts de transaction.

In [ ]:
class Backtester:
    def __init__(self, data, strategy_name, initial_capital=10000, transaction_cost=0.0001):
        self.data = data.copy()
        self.strategy_name = strategy_name
        self.initial_capital = initial_capital
        self.transaction_cost = transaction_cost
        self.results = None

    def run(self, signals):
        """
        signals: pd.Series avec index timestamp et valeurs {1 (Buy), -1 (Sell), 0 (Cash/Hold)}
        """
        # Alignement des signaux
        self.data['signal'] = signals
        self.data['signal'] = self.data['signal'].shift(1) # On trade à l'ouverture suivante
        self.data['signal'].fillna(0, inplace=True)
        
        # Calcul des retours
        # Si Signal 1 (Long), on gagne si Close > Open (approj). Ici on utilise return_15m (Close to Close)
        # Simplification : Retour = Signal * Market_Return - Cost
        
        market_returns = self.data['return_15m']
        
        # Coûts : à chaque changement de position
        trades = self.data['signal'].diff().abs()
        costs = trades * self.transaction_cost
        
        strategy_returns = (self.data['signal'] * market_returns) - costs
        
        # Capital Curve
        self.data['strategy_returns'] = strategy_returns
        self.data['equity'] = (1 + strategy_returns).cumprod() * self.initial_capital
        self.data['drawdown'] = self.data['equity'] / self.data['equity'].cummax() - 1
        
        return self.data['equity']

    def metrics(self):
        total_return = (self.data['equity'].iloc[-1] / self.initial_capital) - 1
        sharpe = self.data['strategy_returns'].mean() / (self.data['strategy_returns'].std() + 1e-9) * np.sqrt(252 * 96) # M15 -> ~96 bars/day
        max_dd = self.data['drawdown'].min()
        
        return {
            'Strategy': self.strategy_name,
            'Total Return': f"{total_return:.2%}",
            'Sharpe Ratio': f"{sharpe:.2f}",
            'Max Drawdown': f"{max_dd:.2%}",
            'Final Equity': f"{self.data['equity'].iloc[-1]:.2f}"
        }

## 3. Stratégie 1 : Règles (EMA + RSI)
**Logique** :
- **Tendance** : EMA 50 > EMA 200 (Long) / EMA 50 < EMA 200 (Short)
- **Entrée** : RSI < 30 (Oversold -> Achat) / RSI > 70 (Overbought -> Vente)

In [ ]:
def strategy_ema_rsi(df):
    signals = pd.Series(0, index=df.index)
    
    # Conditions (Vectorisé)
    # Note: Assurez-vous que les colonnes 'ema_50', 'ema_200', 'rsi_14' existent (créées dans T05)
    # Si ema_200 n'est pas là, on utilisera ema_20 vs ema_50
    
    # Utilisation des colonnes existantes (d'après run_features_T05.py : ema_20, ema_50, rsi_14)
    
    trend_bull = df['ema_20'] > df['ema_50']
    trend_bear = df['ema_20'] < df['ema_50']
    
    long_entry = trend_bull & (df['rsi_14'] < 40) # Pullback en tendance haussière
    short_entry = trend_bear & (df['rsi_14'] > 60) # Pullback en tendance baissière
    
    signals[long_entry] = 1
    signals[short_entry] = -1
    
    # Forward fill (Hold position until signal change or exit?)
    # Ici on simplifie: on reste en position tant que la condition est vraie puis on sort (0).
    # Ou mieux : on garde la dernière position (trend following)
    signals = signals.replace(0, np.nan).fillna(method='ffill').fillna(0)
    
    return signals

## 4. Stratégie 2 : Machine Learning (Random Forest)
Entraînement sur 2022, Validation 2023 (Optimisation simulée), Test 2024.

In [ ]:
# Features pour le ML
features_cols = ['rsi_14', 'ema_20', 'ema_50', 'atr_14', 'adx_14', 'return_15m', 'rolling_std_20']

# Standardisation
scaler = StandardScaler()
X_train = scaler.fit_transform(train_data[features_cols])
y_train = train_data['target']

# Entraînement
model_rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model_rf.fit(X_train, y_train)

# Prédiction sur Test (2024)
X_test = scaler.transform(test_data[features_cols])
obs_pred = model_rf.predict(X_test)

# Conversion 0/1 -> -1/1 (0 -> Short, 1 -> Long)
signals_ml = pd.Series(np.where(obs_pred == 1, 1, -1), index=test_data.index)

## 5. Exécution des Backtests sur 2024 (Test Set)

In [ ]:
results_list = []

# 1. Règles EMA+RSI
signals_rules = strategy_ema_rsi(test_data)
bt_rules = Backtester(test_data, "EMA + RSI")
equity_rules = bt_rules.run(signals_rules)
results_list.append(bt_rules.metrics())

# 2. Machine Learning RF
bt_ml = Backtester(test_data, "Machine Learning (RF)")
equity_ml = bt_ml.run(signals_ml)
results_list.append(bt_ml.metrics())

# 3. Baseline / RL (Simulée)
# Comme le modèle RL n'est pas entraînable ici (pas de stable-baselines3), on utilise le Buy & Hold comme baseline de référence ultime ou une stratégie aléatoire pour illustrer.
# Note: Dans un environnement complet, on chargerait l'agent PPO ici.
signals_bh = pd.Series(1, index=test_data.index) # Buy & Hold
bt_bh = Backtester(test_data, "Buy & Hold (Baseline)")
equity_bh = bt_bh.run(signals_bh)
results_list.append(bt_bh.metrics())

## 6. Comparaison et Visualisation

In [ ]:
results_df = pd.DataFrame(results_list)
print("### Tableau Comparatif (Test 2024) ###")
display(results_df)

# Plot Equity Curves
plt.figure(figsize=(12, 6))
plt.plot(equity_rules, label='EMA + RSI (Règles)', color='#4E8D7C', alpha=0.8)
plt.plot(equity_ml, label='Random Forest (ML)', color='#E85A4F', alpha=0.8)
plt.plot(equity_bh, label='Buy & Hold', color='gray', linestyle='--', alpha=0.5)

plt.title("Comparaison des Stratégies sur 2024 (Test Set)", fontsize=14)
plt.ylabel("Capital ($)")
plt.xlabel("Temps")
plt.legend()
plt.grid(True, alpha=0.3)
plt.savefig('../artifacts/T09_comparative_equity_2024.png')
plt.show()

## 7. Conclusion

**Analyse des Résultats** :
- La stratégie **Rules-Based (EMA+RSI)** offre souvent un meilleur contrôle du risque (Drawdown) mais peut sous-performer en marché range.
- Le **Machine Learning (RF)** tente de capturer des patrons non-linéaires. S'il surperforme le Buy & Hold en 2024, c'est un signe fort de robustesse.
- Le **Buy & Hold** sert de référence de marché. Si les stratégies actives font moins bien que le Buy & Hold, elles ne valent pas le risque/coût.

**Note sur le RL** : L'environnement `TradingEnv` a été défini et testé dans T08, mais l'entraînement d'un agent PPO performant nécessite des ressources de calcul et des librairies spécifiques (stable-baselines3) non disponibles ici. Pour une implémentation future, l'agent RL utiliserait les mêmes observations que le RF mais optimiserait directement la Reward (PnL ajusté du risque) plutot que la précision directionnelle.